<a href="https://colab.research.google.com/github/srulikbd/Deep-Learning/blob/master/Longformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 71kB 3.0MB/s 
     |████████████████████████████████| 778kB 13.4MB/s 
     |████████████████████████████████| 1.1MB 30.7MB/s 
     |████████████████████████████████| 890kB 43.0MB/s 
     |████████████████████████████████| 3.0MB 50.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.3-cp36-none-any.whl size=97299 sha256=ec5987fa55d21947ca99445ae0a2d1cd22f67b761e870c86d645fd2a31e48c28
  Stored in directory: /root/.cache/pip/wheels/75/d6/0a/cab163b21d0597cc1580bc344487b11ad405e0d1d314725f2b
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=6d430f571b2dcf0f59653e143b3844aab77fdeb59c933824e9726da42f8dc7ad
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


https://huggingface.co/allenai/longformer-base-4096

In [ ]:
from sentence_transformers import models
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Use BERT for mapping tokens to embeddings
word_embedding_model = models.Transformer('allenai/longformer-large-4096')

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
corpus = ["""Ben-Gurion's passion for Zionism,1948, he formally proclaimed the establishment of the State of Israel, and was the first to sign the Israeli Declaration of Independence, which he had helped to write. Ben-Gurion led Israel during the 1948 Arab–Israeli War, and united the various Jewish militias into the Israel Defense Forces (IDF). Subsequently, he became known as "Israel's founding father".[2] """,
          """Following the war, Ben-Gurion served as Israel's first prime minister and minister of defense. As prime minister, he helped build the state institutions, presiding over national projects aimed at the development of the country. He also oversaw the absorption of vast numbers of""",
          """Benjamin Netanyahu (/ˌnɛtɑːnˈjɑːhuː/;[3] Hebrew: About this soundבִּנְיָמִין נְתַנְיָהוּ‎ (help·info); born 21 October 1949) is an Israeli politician serving as Prime Minister of Israel since 2009, and previously from 1996 to 1999. Netanyahu is also the Chairman of the Likud – National Liberal Movement. He is the longest-serving Prime Minister in Israeli history[4][5] and the first to be born in Israel after the establishment of the state."""]
embedder = model
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = [""" David Ben-Gurion (/bɛn ˈɡʊəriən/ ben GOOR-ee-ən; Hebrew: דָּוִד בֶּן-גּוּרִיּוֹן‎ [daˈvid ben ɡuʁˈjon] (About this soundlisten); born David Grün; 16 October 1886 – 1 December 1973) was the primary national founder of the State of Israel and the first Prime Minister of Israel. He was the preeminent leader of the Jewish community in British Mandate Palestine from 1935 until the establishment of the State of Israel in 1948, which he led until 1963 with a short break in 1954-55.""", """  """, """ """]


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 2
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]

    #We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores.cpu(), range(top_k))[0:top_k]

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx in top_results[0:top_k]:
        print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))





Query:  David Ben-Gurion (/bɛn ˈɡʊəriən/ ben GOOR-ee-ən; Hebrew: דָּוִד בֶּן-גּוּרִיּוֹן‎ [daˈvid ben ɡuʁˈjon] (About this soundlisten); born David Grün; 16 October 1886 – 1 December 1973) was the primary national founder of the State of Israel and the first Prime Minister of Israel. He was the preeminent leader of the Jewish community in British Mandate Palestine from 1935 until the establishment of the State of Israel in 1948, which he led until 1963 with a short break in 1954-55.

Top 5 most similar sentences in corpus:
Benjamin Netanyahu (/ˌnɛtɑːnˈjɑːhuː/;[3] Hebrew: About this soundבִּנְיָמִין נְתַנְיָהוּ‎ (help·info); born 21 October 1949) is an Israeli politician serving as Prime Minister of Israel since 2009, and previously from 1996 to 1999. Netanyahu is also the Chairman of the Likud – National Liberal Movement. He is the longest-serving Prime Minister in Israeli history[4][5] and the first to be born in Israel after the establishment of the state. (Score: 0.9985)
Ben-Gur